In [23]:
import pandas as pd
import numpy as np
import os
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS, Qdrant
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain.output_parsers.openai_tools import PydanticToolsParser
from langchain.schema.runnable import RunnableLambda
from langchain_core.pydantic_v1 import BaseModel, Field
import time
from qdrant_client import QdrantClient
from langserve import add_routes
from fastapi import FastAPI
from dotenv import load_dotenv
from pathlib import Path

In [ ]:
import math

In [24]:
# PARAMETERS
MODEL_NAME = "gpt-4o"

dotenv_path = Path('/Users/josebookpro/Desktop/DIPLOMADO_GENAI/.env')
load_dotenv(dotenv_path=dotenv_path)

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
LANGCHAIN_API_KEY = os.getenv('LANGCHAIN_API_KEY')

QDRANT_URL = os.getenv('QDRANT_URL')
QDRANT_API_KEY = os.getenv('QDRANT_API_KEY')

# API MinSal

In [25]:
import requests

# URL de la API
url = "https://midas.minsal.cl/farmacia_v2/WS/getLocalesTurnos.php"

# Realizar la solicitud GET
params = {'local_nombre': 'CRUZ VERDE'}

response = requests.get(url, params=params)

# Comprobar si la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Imprimir la respuesta en formato JSON
    data = response.json()  # Si la respuesta es JSON, de lo contrario usa response.text
    # print(data)
else:
    # Mostrar el código de error si la solicitud falló
    print(f"Error en la solicitud: {response.status_code}")


In [27]:
data = pd.DataFrame(data)

In [30]:
# data.local_nombre.value_counts()
data.shape

(212, 15)

In [51]:
url2 = 'https://midas.minsal.cl/farmacia_v2/WS/getLocales.php?local_nombre=CRUZ+VERDE'

url2 = requests.get(url2, params=params)
url2 = pd.DataFrame(url2.json())

In [52]:
url2.shape

(3223, 15)

# Direccion y distancia

In [41]:
def haversine(lon1, lat1, lon2, lat2):
    R = 6371  # Radio de la Tierra en kilómetros
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.asin(math.sqrt(a))
    
    return R * c

In [57]:
# Punto de referencia (puedes cambiar esto por el punto que desees)
# ref_lat, ref_lng = -33.4473852, -70.623185 # Casa
ref_lng, ref_lat = -33.4473852, -70.623185 # Casa

df_cv = url2[url2.local_nombre=='CRUZ VERDE']

df_cv['local_lat'] = pd.to_numeric(df_cv['local_lat'])
df_cv['local_lng'] = pd.to_numeric(df_cv['local_lng'])

# Calcular la distancia y agregarla como una nueva columna
df_cv['distance_km'] = df_cv.apply(lambda row: haversine(ref_lng, ref_lat, row['local_lng'], row['local_lat']), axis=1)

/var/folders/gm/_g___7y91pxgzn4bpxxz1ltw0000gn/T/ipykernel_71419/3570577373.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cv['local_lat'] = pd.to_numeric(df_cv['local_lat'])
/var/folders/gm/_g___7y91pxgzn4bpxxz1ltw0000gn/T/ipykernel_71419/3570577373.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cv['local_lng'] = pd.to_numeric(df_cv['local_lng'])
/var/folders/gm/_g___7y91pxgzn4bpxxz1ltw0000gn/T/ipykernel_71419/3570577373.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy 

In [58]:
df_cv.sort_values('distance_km', ascending=True).localidad_nombre.value_counts()

localidad_nombre
LAS CONDES         51
PROVIDENCIA        25
ÑUÑOA              23
SANTIAGO ESTE      20
SANTIAGO CENTRO    16
                   ..
CONCHALI            1
PANGUIPULLI         1
LAMPA               1
LOS LAGOS           1
RIO BUENO           1
Name: count, Length: 90, dtype: int64

# Vademécum

In [38]:
import os
import zipfile
import pandas as pd

# Descarga el dataset usando la API de Kaggle
dataset = 'anoopjohny/comprehensive-drug-information-dataset'
os.system(f'kaggle datasets download -d {dataset}')

# Nombre del archivo .zip (debe coincidir con el nombre del archivo descargado)
zip_file = 'comprehensive-drug-information-dataset.zip'

# Verifica si el archivo .zip existe antes de descomprimir
if os.path.exists(zip_file):
    # Descomprimir el archivo descargado
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall('comprehensive-drug-information')  # Extrae el contenido en un nuevo directorio

    print(f"Dataset descomprimido en el directorio: comprehensive-drug-information")

    # Encuentra el archivo CSV dentro del directorio descomprimido
    csv_file = os.path.join('comprehensive-drug-information', 'DrugData.csv')  # Nombre del archivo CSV
    
    # Verifica si el archivo CSV existe antes de cargarlo en un DataFrame
    if os.path.exists(csv_file):
        df = pd.read_csv(csv_file)
    else:
        print(f"No se encontró el archivo {csv_file}")
else:
    print(f"No se encontró el archivo {zip_file}")


Dataset URL: https://www.kaggle.com/datasets/anoopjohny/comprehensive-drug-information-dataset
License(s): ODbL-1.0
comprehensive-drug-information-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Dataset descomprimido en el directorio: comprehensive-drug-information
Datos cargados en el DataFrame:


In [39]:
df

,Drug ID,Drug Name,Generic Name,Drug Class,Indications,Dosage Form,Strength,Route of Administration,Mechanism of Action,Side Effects,Contraindications,Interactions,Warnings and Precautions,Pregnancy Category,Storage Conditions,Manufacturer,Approval Date,Availability,NDC,Price
0,1,Aspirin,Acetylsalicylic Acid,Analgesic,Headache,Tablet,325 mg,Oral,Inhibits prostaglandin synthesis,Stomach irritation,Allergy to NSAIDs,Anticoagulants: increased bleeding risk,Gastric ulcers: risk of bleeding,Category C,Room temperature,PharmaCorp,15-01-2022,OTC,12345678901,5.99
1,2,Amoxicillin,Amoxicillin,Antibiotic,Bacterial Infections,Capsule,500 mg,Oral,Inhibits bacterial cell wall synthesis,Nausea,Allergic reactions,Penicillin: reduced efficacy,Take with food to reduce stomach upset,Category B,Room temperature,MediPharm,20-11-2021,Prescription,23456789012,12.49
2,3,Lisinopril,Lisinopril,Antihypertensive,Hypertension,Tablet,10 mg,Oral,Blocks conversion of angiotensin I to angioten...,Cough,Kidney problems,Diuretics: increased risk of kidney problems,Monitor kidney function regularly,Category D,Room temperature,HealthMeds,05-03-2022,Prescription,34567890123,8.75
3,4,Albuterol,Albuterol,Bronchodilator,Asthma,Inhaler,100 mcg/inhalation,Inhalation,Stimulates beta-2 adrenergic receptors,Tremor,Rapid heart rate,Beta-blockers: decreased effectiveness,Use as needed for acute asthma symptoms,Category C,Room temperature,RespirePharma,10-08-2021,Prescription,45678901234,22.99
4,5,Omeprazole,Omeprazole,Proton Pump Inhibitor,Acid Reflux,Capsule,20 mg,Oral,Reduces stomach acid production,Headache,Abdominal pain,Clopidogrel: reduced effectiveness,Take before a meal,Category C,Room temperature,MediCare,30-05-2022,OTC,56789012345,6.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,216,Loratadine,Loratadine,Antihistamine,Allergies,Tablet,10 mg,Oral,Blocks histamine receptors,Drowsiness (less common),Dry mouth,Use as needed for allergy symptoms,May cause drowsiness: use caution while driving,Category B,Room temperature,AllergyRelief,05-11-2021,OTC,90123456690,4.75
216,217,Carvedilol,Carvedilol,Alpha-Beta Blocker,Hypertension,Tablet,12.5 mg,Oral,Blocks alpha and beta receptors,Dizziness,Low blood pressure,COPD: use with caution,Take with food,Category C,Room temperature,CardioPharm,20-01-2022,Prescription,12345678901,6.50
217,218,Sertraline,Sertraline,Selective Serotonin Reuptake Inhibitor,Depression,Tablet,100 mg,Oral,Inhibits serotonin reuptake,Nausea,Insomnia,Monoamine oxidase inhibitors: serotonin syndro...,Avoid alcohol,Category C,Room temperature,MoodEase,15-12-2021,Prescription,23456789012,7.99
218,219,Duloxetine,Duloxetine,Selective Serotonin-Norepinephrine Reuptake In...,Depression,Tablet,60 mg,Oral,Inhibits serotonin and norepinephrine reuptake,Nausea,Insomnia,Monoamine oxidase inhibitors: serotonin syndro...,Avoid alcohol,Category C,Room temperature,EmotionCare,01-03-2022,Prescription,34567890123,9.50
